In [1]:
from importlib import reload
def reload_all():
    import tsm; reload(tsm)
    import _keys; reload(_keys)
    import blizzard; reload(blizzard)
    import config; reload(config)
    import kvstore; reload(kvstore)
    import snapshot; reload(snapshot)
    import elemental_arbitrage; reload(elemental_arbitrage)
    import bliz_tsm_join; reload(bliz_tsm_join)
    import crafting; reload(crafting)
    import kvstore; reload(kvstore)
    import crafting; reload(crafting)
    import procurement; reload(procurement)
reload_all()
from itables import show

In [2]:
from snapshot import SnapshotProcessor
from tsm import auction_house_snapshot
from config import tsm_region_id
from config import tsm_realm_id
from config import tsm_ah_id
from config import tsm_cache_dir

def tsm_ah_snapper():
    return auction_house_snapshot(tsm_region_id, tsm_realm_id, tsm_ah_id)

tsm_ah_snap = SnapshotProcessor(tsm_ah_snapper, cache_dir=tsm_cache_dir)
tsm_ah = tsm_ah_snap.get(max_age_seconds=3000)

In [3]:
from snapshot import SnapshotProcessor
from blizzard import auction_data
from blizzard import auction_summary
from config import blizzard_realm_id
from config import blizzard_ah_id
from config import blizzard_cache_dir

def bliz_ah_snapper():
    return auction_data(blizzard_realm_id, blizzard_ah_id)

bliz_ah_snap = SnapshotProcessor(bliz_ah_snapper, cache_dir=blizzard_cache_dir)
bliz_ah = bliz_ah_snap.get(max_age_seconds=3000)

In [4]:
from kvstore import InefficientKVStore
from blizzard import ItemLookup
from blizzard import collapse_languages
from config import blizzard_item_cache
from config import blizzard_item_reverse_cache

items = ItemLookup(
    InefficientKVStore(blizzard_item_cache),
    InefficientKVStore(blizzard_item_reverse_cache),
)

In [5]:
from bliz_tsm_join import item_info_getter
item_info = item_info_getter(items, bliz_ah_snap, tsm_ah_snap, max_age_seconds=3000)
item_info_current = item_info_getter(items, bliz_ah_snap, tsm_ah_snap, max_age_seconds=0)

In [33]:
reload_all()

from crafting import Recipes
from crafting import purchase_price
from crafting import crafting_price
from crafting import gold


r = Recipes(items)
i = r.ingredients

def buy(item, price_method=gold("realm_market_value")):
    return purchase_price(item_info, item, price_method)


def buy_now(item):
    return buy(item, price_method=gold("min"))


def buy_market(item):
    return buy(item, price_method=gold("realm_market_value"))


recipe_listing = """
2 flask of the frost wyrm
5 icethorn + 5 lichbloom + frost lotus + enchanted vial

2 flask of endless rage
7 lichbloom + 3 goldclover + frost lotus + enchanted vial

2 flask of stoneblood
7 lichbloom + 3 crystallized life + frost lotus + enchanted vial

firecracker salmon
glacial salmon + northern spices

elixir of greater agility
sungrass + goldthorn + crystal vial

titanium bar
8 saronite bar

titanium bar
2 titanium ore

titansteel bar
3 titanium bar + eternal fire + eternal earth + eternal shadow

indestructible potion
2 icethorn + imbued vial

saronite bar
2 saronite ore

potion of speed
2 adder's tongue + pygmy oil + imbued vial

potion of wild magic
2 lichbloom + pygmy oil + imbued vial

titansteel spellblade
6 titansteel bar + 6 infinite dust + 2 frozen orb

titansteel bar
3 titanium bar + eternal fire + eternal earth + eternal shadow

glyph of wild growth
resilient parchment + ethereal ink

spellweave robe
8 spellweave + 6 bolt of imbued frostweave + eternium thread + frozen orb

spellweave
bolt of imbued frostweave + 2 eternal fire

bolt of imbued frostweave
2 bolt of frostweave + 2 infinite dust

bolt of frostweave
5 frostweave cloth

earthsiege diamond
dark jade + huge citrine + eternal fire

skyflare diamond
bloodstone + chalcedony + eternal air

noise machine
2 froststeel tube + 2 overcharged capacitor + 8 handful of cobalt bolts

froststeel tube
8 cobalt bar + crystallized water

overcharged capacitor
4 cobalt bar + crystallized earth

handful of cobalt bolts
2 cobalt bar

cobalt bar
cobalt ore

mechanized snow goggles
8 saronite bar + 2 borean leather + eternal shadow

saronite bar
2 saronite ore

wormhole generator: northrend
8 titanium bar + 2 eternal shadow + 2 eternal fire + 2 eternal water + 2 eternal air
"""

reversible_recipes = """
eternal shadow
10 crystallized shadow

eternal water
10 crystallized water

eternal fire
10 crystallized fire

eternal air
10 crystallized air

eternal life
10 crystallized life

greater planar essence
3 lesser planar essence

greater cosmic essence
3 lesser cosmic essence
"""

r_out = None
for line in recipe_listing.splitlines():
    if line.strip() and not line.strip().startswith("#"):
        if r_out:
            r.recipe_from_strings(r_out, line.strip())
            r_out = None
        else:
            r_out = line.strip()

            
r_out = None
for line in reversible_recipes.splitlines():
    if line.strip() and not line.strip().startswith("#"):
        if r_out:
            r.recipe_from_strings(r_out, line.strip())
            r.recipe_from_strings(line.strip(), r_out)
            r_out = None
        else:
            r_out = line.strip()

from procurement import ProcurementPlanner
from procurement import topk_procurements
pp = ProcurementPlanner(items, item_info, r, approaches=["ah_buy_now", "ah_buy_market", "craft"])

In [7]:
from elemental_arbitrage import print_elemental_arbitrage_report

print_elemental_arbitrage_report(tsm_ah, items)

DEFINITELY   SB eternal-life (unit profit: 37538,headroom: 7197)
SKIP        eternal-earth
SKIP        eternal-air
DEFINITELY   BS eternal-fire (unit profit: 22719,headroom: 1543)
DEFINITELY   BS eternal-water (unit profit: 50356,headroom: 4219)
DEFINITELY   BS eternal-shadow (unit profit: 46325,headroom: 2922)
SKIP        primal-life
SKIP        primal-earth
YES          SB primal-air (unit profit: 109335,headroom: 231)
SKIP        primal-fire
SKIP        primal-water
SKIP        primal-shadow
SKIP        essence-eternal
YES          BS essence-cosmic (unit profit: 43590,headroom: 543)
SKIP        essence-nether
DEFINITELY   BS essence-planar (unit profit: 172051,headroom: 838)
SKIP        essence-astral
SKIP        essence-magic
SKIP        essence-mystic


In [8]:
from cytoolz import groupby
import pandas as pd

df_sales = pd.read_csv("Accounting_Earthfury_sales.csv")
df_sales["mode"] = "sold"
df_purchases = pd.read_csv("Accounting_Earthfury_purchases.csv")
df_purchases["mode"] = "purchased"
df_expired = pd.read_csv("Accounting_Earthfury_expired.csv")
df_expired["mode"] = "expired"
df_canceled = pd.read_csv("Accounting_Earthfury_canceled.csv")
df_canceled["mode"] = "canceled"
df_all = pd.concat([df_sales, df_purchases, df_expired, df_canceled])

entries = list(df_all.transpose().to_dict().values())
grouped = groupby(lambda entry: entry["itemString"], entries)
names = {entry["itemString"]: entry["itemName"] for entry in entries}
nets = {
    k: sum(
        (
            vv["price"]*vv["quantity"] if vv["mode"] == "sold" else
            -vv["price"]*vv["quantity"] if vv["mode"] == "purchased" else
            0
        ) for vv in v
    )
    for (k, v) in grouped.items()
}
ah_events = {k: [vv["mode"] for vv in v if vv["source"] != "Vendor"] for (k, v) in grouped.items()}
sale_rate = {
    k: v.count("sold") / (v.count("sold") + v.count("expired") + v.count("canceled"))
    for (k, v) in ah_events.items()
    if v.count("sold") > 1
}
agg = {k: {"name": names[k], "net": nets[k], "rate": sale_rate[k], "num": len(ah_events[k])} for k in sale_rate}
show(pd.DataFrame(sorted(
    [v for v in agg.values() if v["num"] > 4],
    key=lambda entry: entry["net"],
    reverse=True,
)))

<ipython-input-8-e5b0eba07282>:14: UserWarning: DataFrame columns are not unique, some columns will be omitted.
  entries = list(df_all.transpose().to_dict().values())


In [9]:
import pickle
from cytoolz import get as pick_from_dict
from cytoolz import sliding_window
import glob
import itertools
from blizzard import auction_summary
import re
import numpy as np
import datetime
np.set_printoptions(suppress=True)

_names = """
flask of the frost wyrm
flask of endless rage
flask of stoneblood
titanium bar
indestructible potion
eternal shadow
oily blackmouth
elixir of mighty thoughts
frostweave cloth
wildvine
felcloth
lesser healing potion
runic mana potion
greater planar essence
netherweave cloth
greater healing potion
crystallized water
purple lotus
bruiseweed
arcane dust
tiger lily
crystallized fire
major healing potion
potion of speed
potion of wild magic
lichbloom
pygmy oil
adder's tongue
imbued vial
icethorn
frost lotus
enchanted vial
goldclover
crystallized life
eternal life
deadnettle
elixir of accuracy
lesser planar essence
eternal fire
eternal water
chaotic skyflare diamond
skyflare diamond
potent monarch topaz
monarch topaz
runed scarlet ruby
scarlet ruby
purified twilight opal
twilight opal
potion of speed
potion of wild magic
lichbloom
frostweave cloth
infinite dust
greater cosmic essence
dream shard
eternal belt buckle
titansteel bar
titansteel spellblade
mageweave cloth
rough stone
copper bar
coarse stone
linen cloth
bronze bar
heavy stone
moss agate
wool cloth
medium leather
steel bar
solid stone
mithril bar
mageweave cloth
dense stone
thorium bar
runecloth
fel iron bar
mote of earth
mote of fire
netherweave cloth
adamantite bar
cobalt bar
frostweave cloth
crystallized water
crystallized earth
crystallized fire
borean leather
eternal shadow
saronite bar
""".strip().splitlines()

desired_names = list(set(_names))
desired_ids = [items.get_id(name) for name in desired_names if not name.startswith("#")]


date_re = re.compile(r"202.-..-..T..-..-..")
def _timestamp_from_name(name):
    cap = date_re.search(name).group(0)
    return datetime.datetime.strptime(cap, "%Y-%m-%dT%H-%M-%S").timestamp()


by_item = {}
timeseries = []

for snapshot in sorted(glob.glob("bliz-ah/*")):
    with open(snapshot, "rb") as f:
        entries = pickle.load(f).values()
        timeseries.extend(
            [
                {"timestamp": _timestamp_from_name(snapshot), "item_name": items.get_name(item_id), **auction_summary(item_data)}
                for (item_id, item_data) in zip(desired_ids, entries)
            ]
        )
        for (item_id, item_data) in zip(desired_ids, entries):
            name = items.get_name(item_id)
            by_item[name] = by_item.get(name, []) + [{"timestamp": date_re.search(snapshot).group(0), "item_name": items.get_name(item_id), **auction_summary(item_data)}]

def timeseries_of(key, default=np.nan):
    def _timeseries_of(item_name):
        return np.array([v.get(key, default) for v in by_item[item_name]])
    return _timeseries_of

def omit_outliers(series, sigma=2, window=5):
    mavg = [np.average(w) for w in sliding_window(window, series)]
    std = np.std(mavg)
    return np.array([x if np.abs(x) < sigma*std else np.nan for x in mavg])

def avg_finite(arr):
    return np.average(arr[np.isfinite(arr)])

In [10]:
import plotly.express as px
for yvalue in ["num", "min", "p50", "d_num", "d_min", "d_p50"]:
    df = pd.DataFrame(timeseries)
    df = df.sort_values(by="timestamp")

    if yvalue.startswith("d_"):
        dfg = df.pivot(index="timestamp", columns="item_name", values=yvalue[2:])
        dfg = dfg.diff().div(dfg.index.to_series().diff(), axis=0)
    else:
        dfg = df.pivot(index="timestamp", columns="item_name", values=yvalue)

    fig = px.scatter(dfg, title=yvalue, log_y=False)
    fig.write_html(f"{yvalue}.html")

In [11]:
def min_series(x):
    return omit_outliers(timeseries_of("avg_sell")(x), sigma=0.5, window=10)

wild_magic_pot = avg_finite(min_series("potion of wild magic") - (2*min_series("lichbloom") + min_series("pygmy oil") + np.array(1.8)))
speed_pot = avg_finite(min_series("potion of speed") - (2*min_series("adder's tongue") + min_series("pygmy oil") + np.array(1.8)))
wyrm = avg_finite(2*min_series("flask of the frost wyrm") - (5*min_series("icethorn") + 5*min_series("lichbloom") + min_series("frost lotus") + np.array(5)))
rage = avg_finite(2*min_series("flask of endless rage") - (5*min_series("goldclover") + 7*min_series("lichbloom") + min_series("frost lotus") + np.array(5)))
indestructible = avg_finite(2*min_series("indestructible potion") - (2*min_series("icethorn") + np.array(1.8)))

In [12]:
def item_infos(items):
    return pd.DataFrame([item_info(i) for i in items])

In [13]:
ii = item_infos(_names)
show(ii)

In [14]:
topk_procurements(pp, i("earthsiege diamond"))

[[Craft(item=earthsiege diamond, ingredients=eternal fire + huge citrine + dark jade),
  AHBuyNow(item=eternal fire, gold=22.0),
  AHBuyNow(item=dark jade, gold=0.9985),
  AHBuyMarket(item=huge citrine, gold=1.6232)],
 [Craft(item=earthsiege diamond, ingredients=eternal fire + huge citrine + dark jade),
  Craft(item=eternal fire, ingredients=10.0*crystallized fire),
  Craft(item=10.0*crystallized fire, ingredients=eternal fire),
  AHBuyNow(item=eternal fire, gold=22.0),
  AHBuyNow(item=dark jade, gold=0.9985),
  AHBuyMarket(item=huge citrine, gold=1.6232)],
 [Craft(item=earthsiege diamond, ingredients=eternal fire + huge citrine + dark jade),
  AHBuyNow(item=eternal fire, gold=22.0),
  AHBuyNow(item=dark jade, gold=0.9985),
  AHBuyNow(item=huge citrine, gold=1.6539)],
 [Craft(item=earthsiege diamond, ingredients=eternal fire + huge citrine + dark jade),
  Craft(item=eternal fire, ingredients=10.0*crystallized fire),
  Craft(item=10.0*crystallized fire, ingredients=eternal fire),
  AHBu

In [15]:
topk_procurements(pp, i("spellweave robe"))

[[Craft(item=spellweave robe, ingredients=8.0*spellweave + eternium thread + 6.0*bolt of imbued frostweave + frozen orb),
  AHBuyNow(item=frozen orb, gold=63.75),
  Craft(item=8.0*spellweave, ingredients=16.0*eternal fire + 8.0*bolt of imbued frostweave),
  AHBuyNow(item=16.0*eternal fire, gold=352.0),
  Craft(item=14.0*bolt of imbued frostweave, ingredients=28.0*bolt of frostweave + 28.0*infinite dust),
  Craft(item=28.0*bolt of frostweave, ingredients=140.0*frostweave cloth),
  AHBuyNow(item=140.0*frostweave cloth, gold=28.378),
  AHBuyNow(item=28.0*infinite dust, gold=39.199999999999996),
  AHBuyNow(item=eternium thread, gold=3.0)],
 [Craft(item=spellweave robe, ingredients=8.0*spellweave + eternium thread + 6.0*bolt of imbued frostweave + frozen orb),
  AHBuyNow(item=frozen orb, gold=63.75),
  Craft(item=8.0*spellweave, ingredients=16.0*eternal fire + 8.0*bolt of imbued frostweave),
  Craft(item=16.0*eternal fire, ingredients=160.0*crystallized fire),
  Craft(item=160.0*crystallize

In [16]:
topk_procurements(pp, i("skyflare diamond"))

[[Craft(item=skyflare diamond, ingredients=bloodstone + chalcedony + eternal air),
  Craft(item=eternal air, ingredients=10.0*crystallized air),
  AHBuyNow(item=10.0*crystallized air, gold=5.313),
  AHBuyNow(item=chalcedony, gold=1.5809),
  AHBuyMarket(item=bloodstone, gold=1.5821)],
 [Craft(item=skyflare diamond, ingredients=bloodstone + chalcedony + eternal air),
  Craft(item=eternal air, ingredients=10.0*crystallized air),
  AHBuyNow(item=10.0*crystallized air, gold=5.313),
  AHBuyMarket(item=chalcedony, gold=1.5957),
  AHBuyMarket(item=bloodstone, gold=1.5821)],
 [Craft(item=skyflare diamond, ingredients=bloodstone + chalcedony + eternal air),
  Craft(item=eternal air, ingredients=10.0*crystallized air),
  AHBuyNow(item=10.0*crystallized air, gold=5.313),
  AHBuyNow(item=chalcedony, gold=1.5809),
  AHBuyNow(item=bloodstone, gold=1.7197)],
 [Craft(item=skyflare diamond, ingredients=bloodstone + chalcedony + eternal air),
  Craft(item=eternal air, ingredients=10.0*crystallized air),


In [17]:
topk_procurements(pp, i("eternal air"))

[[Craft(item=eternal air, ingredients=10.0*crystallized air),
  AHBuyNow(item=10.0*crystallized air, gold=5.313)],
 [AHBuyNow(item=eternal air, gold=6.1595)],
 [Craft(item=eternal air, ingredients=10.0*crystallized air),
  Craft(item=10.0*crystallized air, ingredients=eternal air),
  AHBuyNow(item=eternal air, gold=6.1595)],
 [Craft(item=eternal air, ingredients=10.0*crystallized air),
  AHBuyMarket(item=10.0*crystallized air, gold=6.822)]]

In [18]:
topk_procurements(pp, i("noise machine"))

[[AHBuyMarket(item=noise machine, gold=27.1009)],
 [AHBuyNow(item=noise machine, gold=29.7997)],
 [Craft(item=noise machine, ingredients=8.0*handful of cobalt bolts + 2.0*overcharged capacitor + 2.0*froststeel tube),
  AHBuyNow(item=8.0*handful of cobalt bolts, gold=11.9968),
  Craft(item=2.0*overcharged capacitor, ingredients=2.0*crystallized earth + 8.0*cobalt bar),
  AHBuyMarket(item=2.0*crystallized earth, gold=1.6),
  Craft(item=24.0*cobalt bar, ingredients=24.0*cobalt ore),
  AHBuyNow(item=24.0*cobalt ore, gold=24.0),
  Craft(item=2.0*froststeel tube, ingredients=2.0*crystallized water + 16.0*cobalt bar),
  Craft(item=2.0*crystallized water, ingredients=0.2*eternal water),
  AHBuyNow(item=0.2*eternal water, gold=0.28196)],
 [Craft(item=noise machine, ingredients=8.0*handful of cobalt bolts + 2.0*overcharged capacitor + 2.0*froststeel tube),
  AHBuyNow(item=8.0*handful of cobalt bolts, gold=11.9968),
  Craft(item=2.0*overcharged capacitor, ingredients=2.0*crystallized earth + 8.0*

In [30]:
topk_procurements(pp, i("mechanized snow goggles"))

[[Craft(item=mechanized snow goggles, ingredients=8.0*saronite bar + 2.0*borean leather + eternal shadow),
  Craft(item=8.0*saronite bar, ingredients=16.0*saronite ore),
  AHBuyNow(item=16.0*saronite ore, gold=8.0),
  AHBuyNow(item=2.0*borean leather, gold=0.0),
  AHBuyNow(item=eternal shadow, gold=2.4999)],
 [Craft(item=mechanized snow goggles, ingredients=8.0*saronite bar + 2.0*borean leather + eternal shadow),
  Craft(item=8.0*saronite bar, ingredients=16.0*saronite ore),
  AHBuyNow(item=16.0*saronite ore, gold=8.0),
  AHBuyNow(item=2.0*borean leather, gold=0.0),
  Craft(item=eternal shadow, ingredients=10.0*crystallized shadow),
  Craft(item=10.0*crystallized shadow, ingredients=eternal shadow),
  AHBuyNow(item=eternal shadow, gold=2.4999)],
 [Craft(item=mechanized snow goggles, ingredients=8.0*saronite bar + 2.0*borean leather + eternal shadow),
  Craft(item=8.0*saronite bar, ingredients=16.0*saronite ore),
  AHBuyNow(item=16.0*saronite ore, gold=8.0),
  AHBuyNow(item=2.0*borean l

In [38]:
earthsiege = topk_procurements(pp, i("earthsiege diamond"), k=10)
skyflare = topk_procurements(pp, i("skyflare diamond"), k=10)

In [53]:
buy_market(i("10 skyflare diamond"))

300.353

In [51]:
transmute_sales = topk_procurements(pp, i("10 skyflare diamond"))
[(procurement._cost(t), t) for t in transmute_sales]

[(84.76,
  [Craft(item=10*skyflare diamond, ingredients=10.0*bloodstone + 10.0*chalcedony + 10.0*eternal air),
   Craft(item=10.0*eternal air, ingredients=100.0*crystallized air),
   AHBuyNow(item=100.0*crystallized air, gold=53.13),
   AHBuyNow(item=10.0*chalcedony, gold=15.809),
   AHBuyMarket(item=10.0*bloodstone, gold=15.821000000000002)]),
 (84.908,
  [Craft(item=10*skyflare diamond, ingredients=10.0*bloodstone + 10.0*chalcedony + 10.0*eternal air),
   Craft(item=10.0*eternal air, ingredients=100.0*crystallized air),
   AHBuyNow(item=100.0*crystallized air, gold=53.13),
   AHBuyMarket(item=10.0*chalcedony, gold=15.956999999999999),
   AHBuyMarket(item=10.0*bloodstone, gold=15.821000000000002)]),
 (86.13600000000001,
  [Craft(item=10*skyflare diamond, ingredients=10.0*bloodstone + 10.0*chalcedony + 10.0*eternal air),
   Craft(item=10.0*eternal air, ingredients=100.0*crystallized air),
   AHBuyNow(item=100.0*crystallized air, gold=53.13),
   AHBuyNow(item=10.0*chalcedony, gold=15.80

In [43]:
buy_market(i("skyflare diamond")) - procurement._cost(skyflare[0])

21.5593

In [44]:
buy_market(i("earthsiege diamond")) - procurement._cost(earthsiege[0])

27.943299999999997

In [41]:
buy_market(i("skyflare diamond"))

30.0353

In [42]:
buy_market(i("earthsiege diamond"))

52.565